In [1]:
import pandas as pd
import re
import random
import opendatasets as od

import torch
import torchtext
import torch.nn as nn

import torch.nn.functional as F
from torch.optim import Adam

from torchtext.data import Field,LabelField,TabularDataset,BucketIterator
from torchtext.vocab import Vocab
from torchtext.vocab import GloVe



c:\Users\thepr\.conda\envs\tf_exec_2.10\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = 'cuda' if torch.cuda.is_available else 'cpu'
print(device)
print(torch.__version__)
print(torchtext.__version__)

cuda
1.11.0+cu113
0.6.0


In [3]:
od.download('https://www.kaggle.com/datasets/clmentbisaillon/fake-and-real-news-dataset')

Skipping, found downloaded files in ".\fake-and-real-news-dataset" (use force=True to force download)


### Importing Dataset

In [4]:
real_df = pd.read_csv('./fake-and-real-news-dataset/True.csv')
fake_df = pd.read_csv('./fake-and-real-news-dataset/Fake.csv')

In [5]:
real_df.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [6]:
fake_df.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


### Preprocessing and Merging

In [7]:
real_df.drop(['date','subject'],axis=1,inplace=True)
fake_df.drop(['date','subject'],axis=1,inplace=True)

In [8]:
real_df['Label'] = 1.0
fake_df['Label'] = 0.0

In [9]:
real_df['TEXT'] = real_df['title']+real_df['text']
real_df.drop(['title','text'],axis=1,inplace=True)

In [10]:
fake_df['TEXT'] = fake_df['title']+fake_df['text']
fake_df.drop(['title','text'],axis=1,inplace=True)

In [11]:
real_df.head(3)

,Label,TEXT
0,1.0,"As U.S. budget fight looms, Republicans flip t..."
1,1.0,U.S. military to accept transgender recruits o...
2,1.0,Senior U.S. Republican senator: 'Let Mr. Muell...


In [12]:
fake_df.head(3)

,Label,TEXT
0,0.0,Donald Trump Sends Out Embarrassing New Year’...
1,0.0,Drunk Bragging Trump Staffer Started Russian ...
2,0.0,Sheriff David Clarke Becomes An Internet Joke...


In [13]:
news_df = pd.concat([fake_df,real_df],ignore_index=True)
news_df

,Label,TEXT
0,0.0,Donald Trump Sends Out Embarrassing New Year’...
1,0.0,Drunk Bragging Trump Staffer Started Russian ...
2,0.0,Sheriff David Clarke Becomes An Internet Joke...
3,0.0,Trump Is So Obsessed He Even Has Obama’s Name...
4,0.0,Pope Francis Just Called Out Donald Trump Dur...
...,...,...
44893,1.0,'Fully committed' NATO backs new U.S. approach...
44894,1.0,LexisNexis withdrew two products from Chinese ...
44895,1.0,Minsk cultural hub becomes haven from authorit...
44896,1.0,Vatican upbeat on possibility of Pope Francis ...


In [14]:
news_df.rename(columns={'Label':'LABEL'},inplace=True)

In [15]:
#news_df.to_csv('news_data.csv')

### NLP Tasks

In [16]:
news_df['TEXT'] = news_df['TEXT'].apply(lambda x:x.lower())
news_df['TEXT'] = news_df['TEXT'].apply(lambda x:re.sub('[^a-zA-z0-9\s]','',x))

In [17]:
TEXT = Field(tokenize='spacy',tokenizer_language='en_core_web_sm')
LABEL = LabelField(dtype=torch.float32)
Fields =[(None, None),('LABEL',LABEL),('TEXT',TEXT)]

In [18]:
news_ds = TabularDataset(
    path='news_data.csv', format='csv',
    skip_header=True, fields=Fields)

In [19]:
train_data, test_data = news_ds.split(
    split_ratio=[0.8, 0.2],
    random_state=random.seed(47))

print(f'Num Train: {len(train_data)}')
print(f'Num Test: {len(test_data)}')

Num Train: 35918
Num Test: 8980


In [20]:
train_data, valid_data = train_data.split(
    split_ratio=[0.85, 0.15],
    random_state=random.seed(47))

print(f'Num Train: {len(train_data)}')
print(f'Num Validation: {len(valid_data)}')

Num Train: 30530
Num Validation: 5388


In [21]:
print(vars(train_data.examples[0]))

{'LABEL': '0', 'TEXT': ['sickening', 'reason', 'cophating', 'racist', '49ers', 'qb', 'colin', 'kaepernick', 'just', 'announced', 'hell', 'now', 'stand', 'for', 'national', 'anthemit', 'looks', 'like', 'veteran', 'san', 'francisco', '49ers', 'quarterback', 'colin', 'kaepernick', 'might', 'be', 'done', 'with', 'his', 'national', 'anthem', 'protests', '  ', 'now', 'that', 'he', 's', 'looking', 'for', 'a', 'new', 'nfl', 'contractthe', 'controversial', 'player', 'will', 'stand', 'for', ' ', 'the', 'starspangled', 'banner', ' ', 'next', 'season', 'according', 'to', 'sources', 'who', 'spoke', 'to', 'espn', 'thursday', 'kaepernick', 'first', 'made', 'headlines', 'in', 'august', '2016', 'for', 'refusing', 'to', 'stand', 'during', 'his', 'team', 's', 'preseason', 'game', 'against', 'the', 'green', 'bay', 'packers', 'citing', 'the', ' ', 'oppression', ' ', 'of', 'black', 'people', 'in', 'the', 'united', 'stateskaepernick', 'explained', 'his', 'refusal', 'to', 'stand', 'for', 'the', 'starspangled'

In [22]:
TEXT.build_vocab(train_data,max_size=205614)
LABEL.build_vocab(train_data)

print("Vocabualry Size is %d" % len(TEXT.vocab))

Vocabualry Size is 205614


In [23]:
print(TEXT.vocab.freqs.most_common(20))

[('the', 687383), ('to', 373633), ('of', 304821), ('a', 279350), ('and', 278517), (' ', 253551), ('in', 240090), ('that', 162418), ('on', 133236), ('for', 121885), ('s', 121271), ('is', 114708), ('he', 91309), ('said', 88706), ('trump', 86253), ('it', 84920), ('with', 81824), ('was', 79164), ('as', 70735), ('his', 66202)]


In [24]:
vocab_dict = TEXT.vocab.stoi


In [25]:
train_dl,valid_dl,test_dl = BucketIterator.splits((train_data,valid_data,test_data),
                                           batch_size=32,
                                           sort_within_batch=False,
                                           sort_key=lambda x: len(x.TEXT),device=device)

### Create Embedding GloVe

In [26]:
# import gensim.downloader as api
# loaded_glove_model = api.load("glove-wiki-gigaword-300")
# loaded_glove_embeddings = loaded_glove_model.vectors

In [27]:
from scripts.GloveEmbeddings import create_custom_glove_embedding

In [28]:
Glove_embeddings = create_custom_glove_embedding(TEXT.vocab,300)

In [29]:
type(Glove_embeddings)

numpy.ndarray

### RNN-LSTM Model Building

In [30]:
class RNN_LSTM(nn.Module):
    '''
    Building an RNN_LSTM model 
    '''
    def __init__(self,input_dim,embedding_dim,hidden_dim,output_dim,num_layers):
        super().__init__()
        self.num_layers = num_layers
        self.input_dim = input_dim
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim

        #Embedding 
        self.embedding_layer = nn.Embedding.from_pretrained(embeddings=torch.FloatTensor(Glove_embeddings),freeze= False)
        
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=num_layers)
        self.fc = nn.Linear(hidden_dim, output_dim)

        self.softmax = nn.Softmax(dim=1)
    
    def forward(self,x):
        embed = self.embedding_layer(x)

        out,(hidden,_) = self.lstm(embed)

        # out = self.lstm(out)
        
        out = self.fc(out[:,-1,:])

        return out,hidden


In [31]:
model = RNN_LSTM(input_dim=len(TEXT.vocab),
                 num_layers=3,
                 embedding_dim=300,
                 hidden_dim=256,
                 output_dim=2)


model = model.to(device)


In [32]:
print(model)

RNN_LSTM(
  (embedding_layer): Embedding(205614, 300)
  (lstm): LSTM(300, 256, num_layers=3)
  (fc): Linear(in_features=256, out_features=2, bias=True)
  (softmax): Softmax(dim=1)
)


In [ ]:
# # Embedding matrix shape
# it = iter(model.parameters())
# next(it)[0].shape

### Training and Evaluation

In [ ]:
# torch.cuda.get_device_properties(0).name

In [33]:
num_epochs = 1
optimizer = Adam(model.parameters(),lr=0.0025)
loss_fn = nn.CrossEntropyLoss()

In [ ]:
print(device)

cuda


In [ ]:
model = model.to(device)

In [ ]:
for epoch in range(num_epochs):
    train_acc = 0.0
    valid_acc = 0.0
    train_pred, train_orig, valid_pred, valid_orig = [], [], [], []

    for i, batch_data in enumerate(train_dl):
        text = batch_data.TEXT.to(device)
        labels = batch_data.LABEL.type(torch.LongTensor)
        labels = labels.to(device)   
        text = text.permute(1,0)
        #forward
        preds,_ = model.forward(text)
        # print(preds.shape)
        # print(labels.shape)
        #loss
        loss = loss_fn(preds,labels)
        #backprop
        optimizer.zero_grad()
        loss.backward()

        #update wts
        optimizer.step()

        output = F.softmax(preds)
        _, pred = torch.max(output, dim=1)
        correct_tensor = pred.eq(labels.data.view_as(pred))
        accuracy = torch.mean(correct_tensor.type(torch.FloatTensor))
        train_acc += accuracy.item() * text.size(0)
        #print("Epoch {} || Accuracy {}".format(epoch+1, train_acc))
        train_pred.extend(pred.cpu().numpy())
        train_orig.extend(labels.cpu().numpy())
        train_acc = train_acc / len(train_dl.dataset)


C:\Users\thepr\AppData\Local\Temp\ipykernel_22244\2453975595.py:24: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  output = F.softmax(preds)


In [ ]:
train_acc

0.000556851878942415

In [ ]:
if device == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_cached(0)/1024**3,1), 'GB')


NVIDIA GeForce RTX 4070 Laptop GPU
Memory Usage:
Allocated: 0.9 GB
Cached:    9.3 GB


c:\Users\thepr\.conda\envs\tf_exec_2.10\lib\site-packages\torch\cuda\memory.py:384: FutureWarning: torch.cuda.memory_cached has been renamed to torch.cuda.memory_reserved
  warnings.warn(


In [ ]:
print(len(train_orig))
len(train_pred)
count = 0
for i in range(len(train_orig)):
    if train_pred[i] == train_orig[i]:
        count += 1

print(count)

30530
15741


In [ ]:
for _, data in enumerate(train_dl):
    input = data.TEXT
    labels = data.LABEL.type(torch.LongTensor)

    print(input, labels)
    print(input.shape)
    break

tensor([[    7,  3427,     7,  ...,  1242,  1524, 29376],
        [  598,  8438,   417,  ...,   176,   557,  9665],
        [  547,  2202,   185,  ...,   204,  4431,     3],
        ...,
        [    1,     1,     1,  ...,     1,     1,     1],
        [    1,     1,     1,  ...,     1,     1,     1],
        [    1,     1,     1,  ...,     1,     1,     1]], device='cuda:0') tensor([0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1,
        0, 1, 0, 1, 0, 1, 1, 1])
torch.Size([1381, 32])
